In [ ]:
def SST_loader(filename, cal_idx, mode, opt_seq, window_sz, opt_autoregress, perc_dim):
    import scipy.io as sio 
    import numpy as np
    

    ## loading data
    load_data=sio.loadmat(filename) 
    X_load = load_data['X']
    y_load = load_data['y']    
    n_model = X_load.shape[1]
    n_time, n_lead = y_load.shape
    n_member = []
    for m in range(n_model):
        n_member.append(X_load[0,m].shape[-1])
        if X_load[0,m].shape[0] != n_time or X_load[0,m].shape[1] != n_lead:
            print('!!!')

    y_month_id = np.nan*np.ones(shape=[n_time,n_lead])
    for t in range(n_time):
        for l in range(n_lead):
            y_month_id[t,l] = (t+l)%12

    X_month_id = []
    for m in range(n_model):
        temp = np.nan*np.ones(shape=[n_time,n_lead,n_member[m]])
        for t in range(n_time):
            for l in range(n_lead):
                temp[t,l,:] = (t+l)%12
        X_month_id.append(temp)



    ## deseason
    y_monthlymean = np.nan*np.ones(shape=[12])
    y_monthlystd = np.nan*np.ones(shape=[12])    
    cal_data = y_load[cal_idx,0]    # set used to calculate mean and std  
    cal_month_id = y_month_id[cal_idx,0]
    for month in range(12):
        y_monthlymean[month] = np.mean((cal_data[cal_month_id==month]),0)
        y_monthlystd[month] = np.std((cal_data[cal_month_id==month]),0)

    for month in range(12):
        y_mean = y_monthlymean[month]
        y_std = y_monthlystd[month]
        y_load[y_month_id==month] = y_load[y_month_id==month]-y_mean

    for month in range(12):
        X_mean = y_monthlymean[month]
        X_std = y_monthlystd[month]
        for m in range(n_model):
            X_load[0,m][X_month_id[m]==month] = X_load[0,m][X_month_id[m]==month]-X_mean



    # load data
    y = y_load    

    X_raw = []
    for m in range(n_model):
        X_raw.append(X_load[0,m])


    if mode == 'mean':
        n_dim = n_model
        X = np.zeros(shape=(n_time,n_lead,1,0))
        for m in range(n_model):
            temp = np.mean(X_raw[m],axis=2,keepdims=True)
            temp = temp.reshape(n_time,n_lead,1,1)
            X = np.concatenate((X,temp),3)


    if mode == 'model':
        n_dim = n_member
        X = np.nan*np.ones(shape=(n_time,n_lead,n_model,max(n_member)))
        for m in range(n_model):
#             temp = np.sort(X_raw[m],axis=2)
            X[:,:,m,0:n_member[m]] = X_raw[m]



    if opt_autoregress:
        z = np.nan*np.ones(shape=[n_time,n_lead,1,X.shape[-1]])
        for l in range(n_lead):
            z[:,l,0,0] = np.concatenate((np.zeros(shape=[1]),y[0:-1,0]),0)
        X = np.concatenate((X,z),2)
        n_model += 1




    if opt_seq:
        X_pad = np.concatenate((np.zeros(shape=[window_sz-1,n_lead,X.shape[-2],X.shape[-1]]),X),axis=0)  # padding
        y_pad = np.concatenate((np.zeros(shape=[window_sz-1,n_lead]),y),axis=0)  # padding

        seq_X = np.nan*np.ones(shape=[n_time,window_sz,n_lead,X.shape[-2],X.shape[-1]])
        seq_y = np.nan*np.ones(shape=[n_time,window_sz,n_lead])
        for t in range(n_time):
            seq_X[t,:,:,:,:] = X_pad[t:t+window_sz,:,:,:]
        for t in range(n_time):
            seq_y[t,:,:] = y_pad[t:t+window_sz,:]
   
        return seq_X, seq_y, n_dim

    return X, y, n_dim
